# Calculating Polygenic Risk Scores (PRS) using the Classic PRS Method

Follows the steps outlined in Choi et al. to calculate PRS, converted to run in Python using pandas and plink.
1. Choi, S.W., Mak, T.S. & O’Reilly, P.F. Tutorial: a guide to performing polygenic risk score analyses. Nat Protoc (2020). https://doi.org/10.1038/s41596-020-0353-1


### Import Necessary Packages

In [81]:
import numpy as np
import pandas as pd
import gzip
import hashlib
import os
from pandas_plink import read_plink
import subprocess

## Base Data
### Validate the base data file using an MD5 checksum

In [82]:
def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# As stated in https://choishingwan.github.io/PRS-Tutorial/base/#file-transfer
expected_md5 = 'a2b15fb6a2bbbe7ef49f67959b43b160'
# Height base data file location
file_path = 'Height.gwas.txt.gz'

calculated_md5 = calculate_md5(file_path)
print(f"Calculated MD5: {calculated_md5}")

if calculated_md5 == expected_md5:
    print("MD5 checksum verified!")
else:
    print(f"Checksum does not match. Expected {expected_md5}, got {calculated_md5}.")

Calculated MD5: a2b15fb6a2bbbe7ef49f67959b43b160
MD5 checksum verified!


### If checksum is good, import and filter the data
Filters out SNPs with low INFO and MAF values, then drops duplicate SNPs, finally removing ambiguous SNPs

In [83]:
def read_gz_file(file_path):
    with gzip.open(file_path, 'rt') as f:
        return pd.read_csv(f, sep='\s+')
    
base_data = read_gz_file(file_path)

# Filter out SNPs with low INFO and MAF values
filtered_base_data = base_data[(base_data['INFO'] > 0.8) & (base_data['MAF'] > 0.01)]
# Count the number of duplicated SNPs
num_duplicates = filtered_base_data.duplicated(subset=['SNP']).sum()
print(f"Dropped {num_duplicates} duplicated SNPs")
# Drop duplicate SNPs
filtered_base_data = filtered_base_data.drop_duplicates(subset=['SNP'])

# Filter out ambiguous SNPs
ambiguous_snps = filtered_base_data[((filtered_base_data['A1'] == 'A') & (filtered_base_data['A2'] == 'T')) |
                                    ((filtered_base_data['A1'] == 'T') & (filtered_base_data['A2'] == 'A')) |
                                    ((filtered_base_data['A1'] == 'G') & (filtered_base_data['A2'] == 'C')) |
                                    ((filtered_base_data['A1'] == 'C') & (filtered_base_data['A2'] == 'G'))]
filtered_base_data = filtered_base_data.drop(ambiguous_snps.index)

output_file_path = './base/Height.QC.gz'
filtered_base_data.to_csv(output_file_path, index=False, sep='\t', compression='gzip')

# Print the number of non-ambiguous SNPs
num_non_ambiguous_snps = len(filtered_base_data)
print(f"There are a total of {num_non_ambiguous_snps} non-ambiguous SNPs")


print(filtered_base_data.head())

Dropped 2 duplicated SNPs
There are a total of 499617 non-ambiguous SNPs
   CHR      BP         SNP A1 A2       N        SE         P        OR  \
0    1  756604   rs3131962  A  G  388028  0.003017  0.483171  0.997887   
1    1  768448  rs12562034  A  G  388028  0.003295  0.834808  1.000687   
2    1  779322   rs4040617  G  A  388028  0.003033  0.428970  0.997604   
3    1  801536  rs79373928  G  T  388028  0.008413  0.808999  1.002036   
4    1  808631  rs11240779  G  A  388028  0.002428  0.590265  1.001308   

       INFO       MAF  
0  0.890558  0.369390  
1  0.895894  0.336846  
2  0.897508  0.377368  
3  0.908963  0.483212  
4  0.893213  0.450410  


Now, the Base Data has been processed.

## Target Data
### Validate the target data files using an MD5 checksum (using the previous function)
Unzip the [target data](https://drive.google.com/file/d/1uhJR_3sn7RA8U5iYQbcmTp6vFdQiF4F2/view?usp=sharing) into a folder named `target`.


In [84]:
# As stated in https://choishingwan.github.io/PRS-Tutorial/target/#file-transfer
expected_checksums = {
    'EUR.bed': '98bcef133f683b1272d3ea5f97742e0e',
    'EUR.bim': '6b286002904880055a9c94e01522f059',
    'EUR.cov': '85ed18288c708e095385418517e9c3bd',
    'EUR.fam': 'e7b856f0c7bcaffc8405926d08386e97',
    'EUR.height': 'dd445ce969a81cded20da5c88b82d4df'
}

for file_name, expected_md5 in expected_checksums.items():
    file_path = os.path.join('target', file_name)
    calculated_md5 = calculate_md5(file_path)
    if calculated_md5 == expected_md5:
        print(f"{file_name}: MD5 checksum verified!")
    else:
        print(f"{file_name}: Checksum does not match. Expected {expected_md5}, got {calculated_md5}.")

EUR.bed: MD5 checksum verified!
EUR.bim: MD5 checksum verified!
EUR.cov: MD5 checksum verified!
EUR.fam: MD5 checksum verified!
EUR.height: MD5 checksum verified!


### Clean the Target Data using `plink` using the Standard GWAS QC Method
`plink.exe` must be in your working directory.

In [85]:
command = [
    './plink', 
    '--bfile', 'target/EUR', 
    '--maf', '0.01',
    '--hwe', '1e-6',
    '--geno', '0.01',
    '--mind', '0.1',
    '--write-snplist',
    '--make-just-fam',
    '--out', 'target/EUR.QC'
]

result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)


Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to target/EUR.QC.log.
Options in effect:
  --bfile target/EUR
  --geno 0.01
  --hwe 1e-6
  --maf 0.01
  --make-just-fam
  --mind 0.1
  --out target/EUR.QC
  --write-snplist

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
14 people removed due to missing genotype data (--mind).
IDs written to target/EUR.QC.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999816.
535

### Remove highly correlated SNPs

In [86]:
command = [
    './plink',
    '--bfile', 'target/EUR',
    '--keep', 'target/EUR.QC.fam',
    '--extract', 'target/EUR.QC.snplist',
    '--indep-pairwise', '200', '50', '0.25',
    '--out', 'target/EUR.QC'
]
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)


Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to target/EUR.QC.log.
Options in effect:
  --bfile target/EUR
  --extract target/EUR.QC.snplist
  --indep-pairwise 200 50 0.25
  --keep target/EUR.QC.fam
  --out target/EUR.QC

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 540534 variants remaining.
--keep: 489 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999919.
540534 variants and 489 p

### Calculate Hetrozygosity rates

In [87]:
command = [
    './plink',
    '--bfile', 'target/EUR',
    '--extract', 'target/EUR.QC.prune.in',
    '--keep', 'target/EUR.QC.fam',
    '--het',
    '--out', 'target/EUR.QC'
]
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)

Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to target/EUR.QC.log.
Options in effect:
  --bfile target/EUR
  --extract target/EUR.QC.prune.in
  --het
  --keep target/EUR.QC.fam
  --out target/EUR.QC

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 489 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 489 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999958.
268457 variants and 489 people pass filters and

### Remove indivduals with F coeffecients that are more than 3 STDevs from the mean

In [88]:
dat = pd.read_csv("./target/EUR.QC.het", sep='\s+')

mean_F = np.mean(dat['F'])
sd_F = np.std(dat['F'])

# Get samples with F coefficient within 3 STDevs of the population mean
valid = dat[(dat['F'] <= mean_F + 3 * sd_F) & (dat['F'] >= mean_F - 3 * sd_F)]

valid_samples = valid[['FID', 'IID']]
valid_samples.to_csv("./target/EUR.valid.sample", sep="\t", index=False)

num_excluded = len(dat) - len(valid)
print(f"Number of samples excluded due to high heterozygosity rate: {num_excluded}")
print(valid_samples.head())

Number of samples excluded due to high heterozygosity rate: 2
       FID      IID
0  HG00096  HG00096
1  HG00097  HG00097
2  HG00099  HG00099
3  HG00101  HG00101
4  HG00102  HG00102


### Check for sex mislabelling

In [89]:
command = [
    './plink',
    '--bfile', 'target/EUR',
    '--extract', 'target/EUR.QC.prune.in',
    '--keep', 'target/EUR.valid.sample',
    '--check-sex',
    '--out', 'target/EUR.QC'
]
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)

Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to target/EUR.QC.log.
Options in effect:
  --bfile target/EUR
  --check-sex
  --extract target/EUR.QC.prune.in
  --keep target/EUR.valid.sample
  --out target/EUR.QC

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 487 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 487 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999958.
268457 variants and 487 people pass

In [90]:
valid = pd.read_csv("./target/EUR.valid.sample", sep='\t')
dat = pd.read_csv("./target/EUR.QC.sexcheck", sep='\s+')

dat_filtered = dat[dat['FID'].isin(valid['FID'])]

# Further filter to include only rows where 'STATUS' is 'OK'
dat_valid = dat_filtered[dat_filtered['STATUS'] == 'OK']

# Select 'FID' and 'IID' columns and save to a file
dat_valid_samples = dat_valid[['FID', 'IID']]
dat_valid_samples.to_csv("./target/EUR.QC.valid", sep='\t', index=False)

num_excluded = len(dat) - len(dat_valid)
print(f"Number of samples excluded due to mismatched Sex information: {num_excluded}")
print(dat_valid_samples.head())

Number of samples excluded due to mismatched Sex information: 4
       FID      IID
0  HG00096  HG00096
1  HG00097  HG00097
2  HG00099  HG00099
3  HG00101  HG00101
4  HG00102  HG00102


### Remove related samples

In [91]:
command = [
    './plink',
    '--bfile', 'target/EUR',
    '--extract', 'target/EUR.QC.prune.in',
    '--keep', 'target/EUR.QC.valid',
    '--rel-cutoff', '0.125',
    '--out', 'target/EUR.QC'
]
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)

Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to target/EUR.QC.log.
Options in effect:
  --bfile target/EUR
  --extract target/EUR.QC.prune.in
  --keep target/EUR.QC.valid
  --out target/EUR.QC
  --rel-cutoff 0.125

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 268457 variants remaining.
--keep: 483 people remaining.
Using up to 23 threads (change this with --threads).
Before main variant filters, 483 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999957.
268457 variants and 483 people pass

### General final QC'ed target data file

In [92]:
command = [
    './plink',
    '--bfile', 'target/EUR',
    '--make-bed',
    '--keep', 'target/EUR.QC.rel.id',
    '--extract', 'target/EUR.QC.snplist',
    # Include if need to strandflip mismatched SNPs in data: '--exclude', 'target/EUR.mismatch',
    # '--a1-allele', 'EUR.a1'
]
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and any errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)

Output:
PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to plink.log.
Options in effect:
  --bfile target/EUR
  --extract target/EUR.QC.snplist
  --keep target/EUR.QC.rel.id
  --make-bed

32682 MB RAM detected; reserving 16341 MB for main workspace.
551892 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 540534 variants remaining.
--keep: 483 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 483 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.999918.
540534 variants and 483 people pass filters and QC.
Note: No phenotype

## Calculate PRS with `plink`
Update the effect size by taking the natural logarithm of the odds ratio (`OR`) column, and add it as a new `BETA` column

In [94]:
def read_gz_file(file_path):
    with gzip.open(file_path, 'rt') as f:
        return pd.read_csv(f, sep='\t')

file_path = './base/Height.QC.gz'

dat = read_gz_file(file_path)

print(dat.head())
# Perform the log transformation on the 'OR' column and create the 'BETA' column
dat['BETA'] = np.log(dat['OR'])

output_file_path = './base/Height.QC.Transformed.gz'
filtered_base_data.to_csv(output_file_path, index=False, sep='\t')

print(dat.head())

   CHR      BP         SNP A1 A2       N        SE         P        OR  \
0    1  756604   rs3131962  A  G  388028  0.003017  0.483171  0.997887   
1    1  768448  rs12562034  A  G  388028  0.003295  0.834808  1.000687   
2    1  779322   rs4040617  G  A  388028  0.003033  0.428970  0.997604   
3    1  801536  rs79373928  G  T  388028  0.008413  0.808999  1.002036   
4    1  808631  rs11240779  G  A  388028  0.002428  0.590265  1.001308   

       INFO       MAF  
0  0.890558  0.369390  
1  0.895894  0.336846  
2  0.897508  0.377368  
3  0.908963  0.483212  
4  0.893213  0.450410  
   CHR      BP         SNP A1 A2       N        SE         P        OR  \
0    1  756604   rs3131962  A  G  388028  0.003017  0.483171  0.997887   
1    1  768448  rs12562034  A  G  388028  0.003295  0.834808  1.000687   
2    1  779322   rs4040617  G  A  388028  0.003033  0.428970  0.997604   
3    1  801536  rs79373928  G  T  388028  0.008413  0.808999  1.002036   
4    1  808631  rs11240779  G  A  388028 